## 분류기

In [114]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [115]:
!pip install konlpy

In [0]:
import pandas as pd
import numpy as np
import konlpy

In [0]:
train = pd.read_csv("/content/drive/My Drive/data/ratings_train.txt", sep="\t") 
test = pd.read_csv("/content/drive/My Drive/data/ratings_test.txt", sep="\t")

In [0]:
class BayesianFilter:
  def __init__(self): # 객체 내에서 전역변수
    self.words = set() # set 자료구조 : 중복불가
    self.word_dict = {} 
    self.category_dict = {}
    self.likelihood_dict = {}

  def split(self, text):
    '''
    split 함수는 문장에서 단어들을 분리하고 구두점, 조사, 어미를 \n
    제거한다.
    =========================================
    parameter \n
    text : (str) 문자열
    '''
    okt = konlpy.tag.Okt()
    malist = okt.pos(text, norm=True, stem=True)
    # print(malist)

    ## 조사 어미 구두점 제거
    result = []
    
    for word in malist:
      if word[1] not in ["Josa", "Eomi", "Punctuation"]:
        result.append(word[0])
    
    return result

  def inc_word(self, word, category):
    '''
    inc_word 함수는 카테고리별로 단어가 등장한 횟수를 계산한다.\n
    word_dict = { \n
      '광고' : {'파격':1, '세일':1, ...}\n
      '중요' : {'일정':1, '확인':1, ...}\n
    }\n
    word_dict는 객체 내에서 global하기 때문에 자유롭게 fit 훈련이 가능하다.
    ==========================================
    parameter \n
    word : (str) 단어 1개
    category : (str) 그 문장의 카테고리
    
    '''
    if category not in self.word_dict: 
      self.word_dict[category] = {}
    
    # 초기화
    for cate in self.word_dict:
      if word not in self.word_dict[cate]:
        # 0이 되는걸 방지하기 위해 1로 초기화 
        self.word_dict[cate][word] = 1
    
    # +1
    self.word_dict[category][word] += 1 
    self.words.add(word) # 알아서 중복이 제거된다.
    
  def inc_category(self, category):
    if category not in self.category_dict:
      self.category_dict[category] = 0
    self.category_dict[category] += 1

  def arr_sum(self, arr):
    res = 0
    for v in arr:
      res += v
    
    return res

  def arr_mul(self, arr):
    res = 1
    for v in arr:
      res *= v

    return res

  def likelihood(self):
    '''
    우도표 제작
    '''
    # 카테고리별 전체 단어수 계산
    word_sum = {}
    for cate in self.word_dict.keys():
      word_counts = self.word_dict[cate].values()
      word_sum[cate] = self.arr_sum(word_counts)
    
    # 카테고리, 단어별 우도 계산
    for cate in self.word_dict.keys():
      self.likelihood_dict[cate] = {}

      for word in self.word_dict[cate]:
        self.likelihood_dict[cate][word] = self.word_dict[cate][word] / word_sum[cate]
    
    return self.likelihood_dict

  def predict(self, text):
    '''
    예측
    '''
    
    # P(label) = n(label) / n(labels)

    # n(labels)
    # 전체 카테고리 갯수
    sums = self.arr_sum(self.category_dict.values())
    
    # P(label)
    # 전체에 대한 각 카테고리 확률 계산
    p_cate = {}
    for cate in self.category_dict.keys():
      p_cate[cate] = self.category_dict[cate]/sums
    

    # 입력 문자열 해체
    words = self.split(text)   

    # 단어들이 나왔을때 각 카테고리에 들어갈 확률 계산
    pred_dict = {}

    for cate in self.word_dict:

      word_vals = []
      for word in words:
        # 학습되지 않은 단어라면 스킵
        if word not in self.likelihood_dict[cate]:
          continue

        val = self.likelihood_dict[cate][word]
        word_vals.append(val)
      
      # 우도 x p(라벨)
      hf = self.arr_mul(word_vals) * p_cate[cate]
      
      pred_dict[cate] = hf

      prediction = list(pred_dict.keys())[np.argmax(list(pred_dict.values()))]      
    
    return prediction, pred_dict

  def fit(self, text, category):
    '''
    fit 함수는 Bayesian Filter를 훈련시킨다.
    ========================================
    parameter \n
    text : (str) 문자열 \n
    category : (str) 중요인지 광고인지 label
    '''
    word_list = self.split(text)
    # print(word_list)

    for word in word_list:
      self.inc_word(word, category)    
    self.inc_category(category)


In [0]:
bf = BayesianFilter()

In [0]:
train = train.fillna('')

train_texts = train['document'].values
train_labels = train['label'].values

train_zip = zip(train_texts, train_labels)
total_train = len(train_texts)

# test
test = test.fillna('')

test_texts = test['document'].values
test_labels = test['label'].values

test_zip = zip(test_texts, test_labels)
total_test = len(test_texts)

## 학습

In [0]:
batch = 150000

for i in range(batch):
  text, category = next(train_zip)
  bf.fit(text, category)

## 우도 계산

In [0]:
bf.likelihood()
print("ok")

ok


In [0]:
bf.predict(test_texts[4])

(0, {0: 7.440320452175902e-54, 1: 3.4465374221996403e-56})

## 테스트 정확도 계산


In [0]:
test_size = 50000
pred = []

# 예측 리스트 생성
for i in range(test_size):
  pred.append(bf.predict(test_texts[i]))

# 정답 갯수 확인
correct = 0
for i in range(test_size):
  if pred[i][0] == test_labels[i]:
    correct += 1

# 정확도
print("정확도:", (correct/test_size)*100)

정확도: 84.19200000000001


## 테스트

In [0]:
for i in [0,2,29,1999,20289,35000]:
  print(test_texts[i])
  print(bf.predict(test_texts[i]))

굳 ㅋ
(1, {0: 2.2221964680313306e-08, 1: 3.9795510356476893e-07})
뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
(0, {0: 1.5355518641514873e-36, 1: 3.469608763707977e-37})
지금까지 본 영화중 마음이 가장 따뜻해지는 영화.
(1, {0: 9.274601414432237e-23, 1: 1.0210002826109496e-19})
초등학교 6학년. 우리들의 일그러진 영웅을 읽고 주제에 대해 글을 써오라는 숙제가 주어졌었고 난 이 소설의 깊이를 전혀 이해 못한채 줄거리만 배껴 써내기 바빴던 기억이 난다. 당연히 이해 못할수 밖에.
(0, {0: 1.8229302592091564e-119, 1: 1.4600798697905398e-119})
성우가 좀 그랬지만 10점
(0, {0: 5.4186010734480904e-15, 1: 8.388040500904165e-16})
감성팔이 10자끝?
(1, {0: 1.1184093053552316e-17, 1: 1.4400381061252936e-17})


In [118]:
test['label'].value_counts()

1    25173
0    24827
Name: label, dtype: int64